In [2]:
# ===================== Cell 1. Setup & Load =====================
import os, gc, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import kagglehub
from glob import glob

# --- Download dataset (cached by kagglehub) ---
path = kagglehub.dataset_download("denismuradyan/dataset-olympics-aidao-yandex-2025")
print("Path to dataset files:", path)

# --- Columns (50) per README; dataset has NO headers ---
COLS = [
    'block_id','frame_idx','E_mu_Z','E_mu_phys_est','E_mu_X','E_nu1_X','E_nu2_X','E_nu1_Z','E_nu2_Z',
    'N_mu_X','M_mu_XX','M_mu_XZ','M_mu_X','N_mu_Z','M_mu_ZZ','M_mu_Z',
    'N_nu1_X','M_nu1_XX','M_nu1_XZ','M_nu1_X','N_nu1_Z','M_nu1_ZZ','M_nu1_Z',
    'N_nu2_X','M_nu2_XX','M_nu2_XZ','M_nu2_X','N_nu2_Z','M_nu2_ZZ','M_nu2_Z',
    'nTot','bayesImVoltage','opticalPower',
    'polarizerVoltages[0]','polarizerVoltages[1]','polarizerVoltages[2]','polarizerVoltages[3]',
    'temp_1','biasVoltage_1','temp_2','biasVoltage_2',
    'synErr','N_EC_rounds','maintenance_flag','estimator_name','f_EC','E_mu_Z_est','R','s','p'
]

# --- Robust CSV selection: pick file with the largest #columns (~50) ---
def count_cols(p):
    try:
        return pd.read_csv(p, header=None, nrows=3).shape[1]
    except Exception:
        return -1

csv_candidates = sorted(glob(os.path.join(path, "*.csv")))
if not csv_candidates:
    raise FileNotFoundError("No CSV files found under dataset path.")

csv_scored = [(p, count_cols(p)) for p in csv_candidates]
csv_scored = [x for x in csv_scored if x[1] >= 40]  # expect ~50
input_csv = sorted(csv_scored, key=lambda x: -x[1])[0][0]
print("Using CSV:", os.path.basename(input_csv))

# --- Read WITHOUT header; keep strings where needed (estimator_name) ---
df = pd.read_csv(input_csv, header=None, names=COLS, low_memory=False)
print("Raw shape:", df.shape)

# --- Sort by (block_id, frame_idx) and add causal order index ---
df = df.sort_values(['block_id','frame_idx']).reset_index(drop=True)
df['_ord_'] = np.arange(len(df))

# --- Window to predict: exactly 2000 rows starting at {block_id=1489460492, frame_idx=99} ---
START_BLOCK, START_FRAME = 1489460492, 99
target_len = 2000

start_pos = df.index[(df.block_id==START_BLOCK) & (df.frame_idx==START_FRAME)]
assert len(start_pos)>0, "Start {block_id, frame_idx} not found in data"
start_ord = int(start_pos.min())
end_ord   = start_ord + target_len - 1
assert end_ord < len(df), "Not enough rows for the 2000-frame window"

target_idx = np.arange(start_ord, end_ord+1)
print("Target window ord:", start_ord, "→", end_ord, " (len:", len(target_idx), ")")

# consistency checks
assert 'E_mu_Z' in df.columns and 'block_id' in df.columns and 'frame_idx' in df.columns

Path to dataset files: /kaggle/input/dataset-olympics-aidao-yandex-2025
Using CSV: frames_errors.csv
Raw shape: (328617, 50)
Target window ord: 223704 → 225703  (len: 2000 )


In [3]:
# ===================== Cell 2. Feature Engineering =====================
import numpy as np
import pandas as pd

def add_group_lags(df, group_cols, col, lags):
    g = df.groupby(group_cols)[col]
    for L in lags:
        df[f'{col}_lag{L}'] = g.shift(L)
    return df

def add_group_ema(df, group_cols, col, alphas):
    # names: <col>_ema{alpha}, e.g. E_mu_Z_ema0.33
    g = df.groupby(group_cols)[col]
    for a in alphas:
        cname = f"{col}_ema{a}"
        df[cname] = g.transform(lambda s: s.ewm(alpha=a, adjust=False).mean())
    return df

def add_group_rollstd(df, group_cols, col, windows):
    for w in windows:
        df[f'{col}_std{w}'] = (
            df.groupby(group_cols)[col]
              .rolling(w, min_periods=2).std()
              .reset_index(level=group_cols, drop=True)
        )
    return df

# Base signals to feature-ize
base_signals = [c for c in ['E_mu_Z','E_mu_phys_est','opticalPower'] if c in df.columns]

# Lags + EMA
for c in base_signals:
    df = add_group_lags(df, ['block_id'], c, lags=[1,2,3,4,5,8,12,16,24,32])
    df = add_group_ema(df, ['block_id'], c, alphas=[0.2, 0.33, 0.5])

# Volatility (rolling std)
df = add_group_rollstd(df, ['block_id'], 'E_mu_Z', windows=[8,16])

# Simple deltas/medians
df['E_mu_Z_diff1'] = df.groupby('block_id')['E_mu_Z'].diff(1)
if 'opticalPower' in df.columns:
    df['opticalPower_diff1'] = df.groupby('block_id')['opticalPower'].diff(1)
else:
    df['opticalPower_diff1'] = 0.0

df['E_mu_Z_med9'] = df.groupby('block_id')['E_mu_Z'].transform(lambda s: s.rolling(9, min_periods=2).median())
df['E_mu_Z_med_dev'] = (df['E_mu_Z'] - df['E_mu_Z_med9']).abs()

# Rolling max/min (lagged)
for w in [8,16]:
    df[f'E_mu_Z_max{w}_lag1'] = df.groupby('block_id')['E_mu_Z'].transform(lambda s: s.rolling(w, min_periods=2).max()).shift(1)
    df[f'E_mu_Z_min{w}_lag1'] = df.groupby('block_id')['E_mu_Z'].transform(lambda s: s.rolling(w, min_periods=2).min()).shift(1)

# Prev flags
df['maint_prev'] = df.groupby('block_id')['maintenance_flag'].shift(1) if 'maintenance_flag' in df.columns else 0
df['synErr_prev'] = df.groupby('block_id')['synErr'].shift(1) if 'synErr' in df.columns else 0
df['N_EC_prev']   = df.groupby('block_id')['N_EC_rounds'].shift(1) if 'N_EC_rounds' in df.columns else 0

# Block-wise seasonality proxies
denom = max(1, df['frame_idx'].max())
df['frame_sin'] = np.sin(2*np.pi*df['frame_idx']/denom)
df['frame_cos'] = np.cos(2*np.pi*df['frame_idx']/denom)

# Category for block id
df['block_id_str'] = df['block_id'].astype(str)

# Clean NaNs/Infs for ML features
for c in ['E_mu_Z_diff1','opticalPower_diff1','E_mu_Z_med9','E_mu_Z_med_dev',
          'E_mu_Z_max8_lag1','E_mu_Z_min8_lag1','E_mu_Z_max16_lag1','E_mu_Z_min16_lag1',
          'E_mu_Z_std8','E_mu_Z_std16']:
    if c in df.columns:
        df[c] = df[c].fillna(0.0)

# Feature list (exclude outputs/targets/ids)
exclude_cols = set(['R','s','p','f_EC','estimator_name','E_mu_Z_est','_ord_'])
feature_cols = [c for c in df.columns if c not in exclude_cols and c not in ['block_id','frame_idx']]
df[feature_cols] = df[feature_cols].replace([np.inf,-np.inf], np.nan).fillna(0.0)

print("Feature count:", len(feature_cols))

Feature count: 97


In [4]:
# ===================== Cell 3. Quantile CatBoost for E_mu_Z =====================
import os
from catboost import CatBoostRegressor, Pool

# Train/cal splits: all rows strictly before target window for training;
# keep a small tail as calibration set
train_mask = df['_ord_'] < start_ord
train_idx_all = df.index[train_mask]

cal_tail = min(500, len(train_idx_all)//5 if len(train_idx_all)>1000 else max(100, len(train_idx_all)//3))
cal_idx = train_idx_all[-cal_tail:] if len(train_idx_all) > cal_tail else train_idx_all
fit_idx = train_idx_all.difference(cal_idx)

# Build design matrices
def make_X(idx):
    X = df.loc[idx, feature_cols + ['block_id_str']].copy()
    X = X.loc[:, ~X.columns.duplicated()].copy()
    X['block_id_str'] = X['block_id_str'].astype('category')
    return X

X_fit_df = make_X(fit_idx)
X_cal_df = make_X(cal_idx)
X_tst_df = make_X(target_idx)

y_fit = df.loc[fit_idx, 'E_mu_Z'].astype(float).to_numpy()
y_cal = df.loc[cal_idx, 'E_mu_Z'].astype(float).to_numpy()

task_type = 'GPU' if os.path.exists('/usr/bin/nvidia-smi') else 'CPU'
cat_idx = [X_fit_df.columns.get_loc('block_id_str')]

train_pool = Pool(X_fit_df, y_fit, cat_features=cat_idx)
cal_pool   = Pool(X_cal_df, y_cal, cat_features=cat_idx)
tst_pool   = Pool(X_tst_df,            cat_features=cat_idx)

q_list   = [0.50, 0.80, 0.90, 0.95]
models   = {}
qhat     = {}   # preds on target
qhat_cal = {}   # preds on cal

for q in q_list:
    mdl = CatBoostRegressor(
        loss_function=f'Quantile:alpha={q}',
        iterations=3000,
        depth=6,
        learning_rate=0.05,
        l2_leaf_reg=3.0,
        random_state=42,
        task_type=task_type,
        verbose=250,
        early_stopping_rounds=200
    )
    mdl.fit(train_pool, eval_set=cal_pool, use_best_model=True)
    models[q]   = mdl
    qhat_cal[q] = mdl.predict(cal_pool)
    qhat[q]     = mdl.predict(tst_pool)

print("Quantile models trained.")

0:	learn: 0.0021181	test: 0.0010694	best: 0.0010694 (0)	total: 185ms	remaining: 9m 15s
250:	learn: 0.0001266	test: 0.0000553	best: 0.0000553 (250)	total: 24.5s	remaining: 4m 27s
500:	learn: 0.0000889	test: 0.0000326	best: 0.0000326 (500)	total: 49s	remaining: 4m 4s
750:	learn: 0.0000769	test: 0.0000248	best: 0.0000248 (750)	total: 1m 13s	remaining: 3m 40s
1000:	learn: 0.0000710	test: 0.0000217	best: 0.0000217 (1000)	total: 1m 38s	remaining: 3m 17s
1250:	learn: 0.0000666	test: 0.0000195	best: 0.0000195 (1250)	total: 2m 3s	remaining: 2m 53s
1500:	learn: 0.0000636	test: 0.0000185	best: 0.0000185 (1500)	total: 2m 29s	remaining: 2m 28s
1750:	learn: 0.0000607	test: 0.0000176	best: 0.0000176 (1750)	total: 2m 57s	remaining: 2m 6s
2000:	learn: 0.0000583	test: 0.0000170	best: 0.0000170 (2000)	total: 3m 23s	remaining: 1m 41s
2250:	learn: 0.0000569	test: 0.0000166	best: 0.0000166 (2250)	total: 3m 49s	remaining: 1m 16s
2500:	learn: 0.0000553	test: 0.0000163	best: 0.0000163 (2500)	total: 4m 15s	rema

In [ ]:
# # ===================== Cell 3. Quantile XGBoost for E_mu_Z =====================
# import os
# import numpy as np
# import xgboost as xgb

# # --- 1. Train/cal split ---
# train_mask = df['_ord_'] < start_ord
# train_idx_all = df.index[train_mask]

# cal_tail = min(500, len(train_idx_all)//5 if len(train_idx_all)>1000 else max(100, len(train_idx_all)//3))
# cal_idx = train_idx_all[-cal_tail:] if len(train_idx_all) > cal_tail else train_idx_all
# fit_idx = train_idx_all.difference(cal_idx)

# # --- 2. Build design matrices ---
# def make_X(idx):
#     X = df.loc[idx, feature_cols + ['block_id_str']].copy()
#     X = X.loc[:, ~X.columns.duplicated()].copy()
#     # One-hot для категориальной переменной, т.к. XGBoost не принимает category напрямую
#     X = pd.get_dummies(X, columns=['block_id_str'], drop_first=True)
#     return X

# X_fit_df = make_X(fit_idx)
# X_cal_df = make_X(cal_idx)
# X_tst_df = make_X(target_idx)

# y_fit = df.loc[fit_idx, 'E_mu_Z'].astype(float).to_numpy()
# y_cal = df.loc[cal_idx, 'E_mu_Z'].astype(float).to_numpy()

# # --- 3. DMatrix для XGBoost ---
# dtrain = xgb.DMatrix(X_fit_df, label=y_fit)
# dcal   = xgb.DMatrix(X_cal_df, label=y_cal)
# dtest  = xgb.DMatrix(X_tst_df)

# # --- 4. Определяем quantile objective ---
# def quantile_loss(alpha):
#     """Возвращает custom objective для quantile loss."""
#     def _quantile_obj(y_pred, dtrain):
#         y_true = dtrain.get_label()
#         error = y_true - y_pred
#         grad = np.where(error < 0, -alpha, 1 - alpha)
#         hess = np.ones_like(grad)  # постоянный гессиан
#         return grad, hess
#     return _quantile_obj

# # --- 5. Метрика для валидации ---
# def quantile_eval(alpha):
#     def _eval(y_pred, dtrain):
#         y_true = dtrain.get_label()
#         error = y_true - y_pred
#         q_loss = np.mean(np.maximum(alpha * error, (alpha - 1) * error))
#         return f'quantile-{alpha:.2f}', q_loss
#     return _eval

# # --- 6. Параметры XGBoost ---
# base_params = {
#     'tree_method': 'gpu_hist',    # используем GPU
#     'predictor': 'gpu_predictor',
#     'max_depth': 6,
#     'eta': 0.05,
#     'lambda': 3.0,
#     'subsample': 1.0,
#     'colsample_bytree': 1.0,
#     'seed': 42,
#     'objective': 'reg:squarederror'  # заглушка (реальный objective кастомный)
# }

# num_boost_round = 3000
# early_stopping_rounds = 200

# # --- 7. Обучаем модель для каждого квантиля ---
# q_list   = [0.50, 0.80, 0.90, 0.95]
# models   = {}
# qhat     = {}   # preds on target
# qhat_cal = {}   # preds on cal

# for q in q_list:
#     print(f"\n===== Training XGBoost Quantile alpha={q} =====")
#     bst = xgb.train(
#         base_params,
#         dtrain,
#         num_boost_round=num_boost_round,
#         evals=[(dcal, 'cal')],
#         obj=quantile_loss(q),
#         feval=quantile_eval(q),
#         early_stopping_rounds=early_stopping_rounds,
#         verbose_eval=250
#     )
#     models[q]   = bst
#     qhat_cal[q] = bst.predict(dcal)
#     qhat[q]     = bst.predict(dtest)

# print("✅ Quantile XGBoost models trained on GPU.")

In [5]:
# ===================== Cell 4. Per-block SHIFT+SCALE calibration of quantiles =====================
from sklearn.isotonic import IsotonicRegression

# If for any reason cal_idx is small, fallback to last few thousand rows before start
if len(cal_idx) < 200:
    bt_len = 3000
    cal_idx = np.arange(max(0, start_ord - bt_len), start_ord)

blocks_cal = df.loc[cal_idx, 'block_id'].astype(int).to_numpy()
y_cal      = df.loc[cal_idx, 'E_mu_Z'].astype(float).to_numpy()
q50_cal    = np.asarray(qhat_cal[0.50], dtype=float)

iso_by_block = {}
scale_by_block = {}

dfc = pd.DataFrame({'block_id': blocks_cal, 'y': y_cal, 'q50': q50_cal})
min_pts = 60

for bid, grp in dfc.groupby('block_id'):
    if len(grp) < min_pts:
        continue
    # 1) isotonic: q50 -> y
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(grp['q50'].values, grp['y'].values)
    iso_by_block[int(bid)] = iso
    # 2) scale tails: IQR ratio
    iqr_y = np.subtract(*np.percentile(grp['y'].values, [75, 25]))
    iqr_q = np.subtract(*np.percentile(grp['q50'].values, [75, 25]))
    scale = 1.0 if iqr_q<=1e-8 else float(np.clip(iqr_y/iqr_q, 0.7, 1.6))
    scale_by_block[int(bid)] = scale

g_scale = float(np.median(list(scale_by_block.values()))) if len(scale_by_block)>0 else 1.0

# Apply to target: for each quantile q use: iso(q50) + scale*(q - q50)
blocks_t = df.loc[target_idx, 'block_id'].astype(int).to_numpy()
q50_t = np.asarray(qhat[0.50], dtype=float)

Eq = {}
for q in q_list:
    base = np.asarray(qhat[q], dtype=float)
    out  = np.empty_like(base)
    for i, bid in enumerate(blocks_t):
        iso = iso_by_block.get(int(bid), None)
        sc  = scale_by_block.get(int(bid), g_scale)
        mu  = float(iso.predict([q50_t[i]])[0]) if iso is not None else q50_t[i]
        out[i] = mu + sc * (base[i] - q50_t[i])
    Eq[q] = np.clip(out, 1e-6, 0.4999)

print("Shift+scale calibration applied.")

Shift+scale calibration applied.


In [9]:
# ===================== Cell 5. Adaptive alpha & final E_mu_Z_est =====================
# Build alpha (risk-averse on volatility/trouble flags)
vol8_t  = df.loc[target_idx, 'E_mu_Z_std8'].fillna(0.0).to_numpy()
vol16_t = df.loc[target_idx, 'E_mu_Z_std16'].fillna(0.0).to_numpy()
trend_up= (df.loc[target_idx, 'E_mu_Z_diff1'].fillna(0.0).to_numpy() > 0).astype(float)
flags   = ((df.loc[target_idx, 'maintenance_flag'].fillna(0)==1).astype(float)
          + (df.loc[target_idx, 'synErr_prev'].fillna(0)>0).astype(float)
          + (df.loc[target_idx, 'N_EC_prev'].fillna(0)>1).astype(float))

alpha = np.clip(0.865
                + 0.35*np.clip(vol8_t,  0, 0.05)
                + 0.25*np.clip(vol16_t, 0, 0.05)
                + 0.02*trend_up
                + 0.01*np.clip(flags,0,2), 0.83, 0.93)

# --- robust: convert possible Series with nonzero index to numpy arrays ---
def _as_np(x):
    if isinstance(x, pd.Series):
        return x.to_numpy(dtype=float)
    return np.asarray(x, dtype=float)

# Fetch calibrated quantiles safely (keys can be 0.5 or 0.50 etc.)
q50 = _as_np(Eq.get(0.5,  Eq.get(0.50)))
q80 = _as_np(Eq.get(0.8,  Eq.get(0.80)))
q90 = _as_np(Eq.get(0.9,  Eq.get(0.90)))
q95 = _as_np(Eq.get(0.95, Eq.get(0.95)))

a = np.asarray(alpha, dtype=float)

# --- vectorized piecewise-linear interpolation over quantiles ---
E_pred_final = np.empty_like(q50, dtype=float)

m1 = (a <= 0.50)
m2 = (a > 0.50) & (a <= 0.80)
m3 = (a > 0.80) & (a <= 0.90)
m4 = (a > 0.90) & (a <= 0.95)
m5 = (a > 0.95)

E_pred_final[m1] = q50[m1]

t = (a[m2] - 0.50) / 0.30
E_pred_final[m2] = q50[m2] + t * (q80[m2] - q50[m2])

t = (a[m3] - 0.80) / 0.10
E_pred_final[m3] = q80[m3] + t * (q90[m3] - q80[m3])

t = (a[m4] - 0.90) / 0.05
E_pred_final[m4] = q90[m4] + t * (q95[m4] - q90[m4])

E_pred_final[m5] = q95[m5]

E_pred_final = np.clip(E_pred_final, 1e-6, 0.4999)
print("E_pred_final stats:", np.round([E_pred_final.min(), E_pred_final.mean(), E_pred_final.max()], 5))

E_pred_final stats: [0.0142  0.0197  0.03649]


In [11]:
# ===================== Cell 6. Backtest headroom tuning (fixed to_numpy) =====================
def H2(x):
    x = np.clip(x, 1e-12, 1-1e-12)
    return -(x*np.log2(x) + (1-x)*np.log2(1-x))

# Backtest window before start
bt_len = 12000
bt_start = max(0, start_ord - bt_len)
bt_idx = np.arange(bt_start, start_ord)
if len(bt_idx) < 2000:
    bt_idx = np.arange(max(0, start_ord-4000), start_ord)

# Historical fEC median for safety baseline
hist_mask = df['_ord_'] < start_ord
fec_hist2 = (1.0 - df.loc[hist_mask, 'R'].astype(float)) / H2(df.loc[hist_mask, 'E_mu_Z'].astype(float))
fec_hist2 = fec_hist2.replace([np.inf, -np.inf], np.nan)

fec_global_med = float(np.clip(np.nanmedian(fec_hist2), 1.06, 1.18))
fec_block_med = (df.loc[hist_mask, ['block_id']]
                   .assign(fec=fec_hist2)
                   .groupby('block_id')['fec'].median()).clip(1.06, 1.20)
# удобно как питоновский dict
fec_block_med_map = {int(k): float(v) for k, v in fec_block_med.items() if pd.notna(v)}

R_GRID = np.round(np.arange(0.50, 0.901, 0.05), 2)

# --- strictly as numpy arrays ---
vol8_b   = df.loc[bt_idx, 'E_mu_Z_std8'].fillna(0.0).to_numpy()
vol16_b  = df.loc[bt_idx, 'E_mu_Z_std16'].fillna(0.0).to_numpy()
trend_b  = (df.loc[bt_idx, 'E_mu_Z_diff1'].fillna(0.0).to_numpy() > 0).astype(float)

maint_b  = (df.loc[bt_idx, 'maintenance_flag'].fillna(0).to_numpy()==1).astype(float)
syn_b    = (df.loc[bt_idx, 'synErr_prev'].fillna(0).to_numpy()>0).astype(float)
nec_b    = (df.loc[bt_idx, 'N_EC_prev'].fillna(0).to_numpy()>1).astype(float)
flags_b  = maint_b + syn_b + nec_b     # numpy array!

E_true_b = df.loc[bt_idx, 'E_mu_Z'].astype(float).to_numpy()
blocks_b = df.loc[bt_idx, 'block_id'].astype(int).to_numpy()

def eval_policy(params):
    base_head, kv8, kv16, ktrend, s_ceiling = params
    R_prev = None
    R_seq, s_seq, Rt_true_seq = [], [], []

    for j in range(len(bt_idx)):
        block = int(blocks_b[j])

        # per-block fec with global fallback
        fec_row = 0.30*float(fec_block_med_map.get(block, fec_global_med)) + 0.70*fec_global_med
        fec_eff = fec_row + 0.06*np.clip(vol8_b[j],0,0.05) + 0.04*np.clip(vol16_b[j],0,0.05)
        if flags_b[j] >= 1:
            fec_eff += 0.01
        fec_eff = float(np.clip(fec_eff, 1.05, 1.20))

        Rt_true = float(np.clip(1.0 - fec_eff*H2(E_true_b[j]), 0.0, 0.95))
        Rt_true_seq.append(Rt_true)

        hr = float(np.clip(base_head + kv8*vol8_b[j] + kv16*vol16_b[j] + ktrend*(trend_b[j]>0), 0.005, 0.030))
        safe = max(0.0, Rt_true - hr)

        r_cands = R_GRID[R_GRID <= safe]
        r = float(r_cands.max()) if len(r_cands) else 0.50

        allow_up2 = (vol8_b[j] < 0.007) and (vol16_b[j] < 0.01) and (trend_b[j] == 0) and (flags_b[j] == 0)
        if R_prev is not None:
            j_prev = int(np.where(R_GRID==np.round(R_prev,2))[0][0])
            j_cur  = int(np.where(R_GRID==np.round(r,2))[0][0])
            j_cur  = max(min(j_cur, j_prev+(2 if allow_up2 else 1)), j_prev-1)
            r      = float(R_GRID[j_cur])

        s = int(np.round(32000*r - 27200*Rt_true))
        s = int(np.clip(s, 0, 4800))
        if s >= s_ceiling:
            pos = int(np.where(R_GRID==np.round(r,2))[0][0])
            if pos>0:
                r2 = float(R_GRID[pos-1])
                s2 = int(np.round(32000*r2 - 27200*Rt_true))
                s2 = int(np.clip(s2, 0, 4800))
                if s2 <= (s_ceiling-80):
                    r, s = r2, s2

        R_seq.append(r); s_seq.append(s); R_prev = r

    R_seq      = np.asarray(R_seq, dtype=float)
    s_seq      = np.asarray(s_seq, dtype=int)
    Rt_true_seq= np.asarray(Rt_true_seq, dtype=float)

    fail_sur = ((R_seq > (Rt_true_seq + 0.003)) | (s_seq > (s_ceiling + 50))).mean()
    score = R_seq.mean() - 6.0*fail_sur - 0.08*(s_seq/4800.0).mean()
    return score, R_seq.mean(), fail_sur

grid = [
    (0.010, 0.30, 0.20, 0.004, 4700),
    (0.010, 0.35, 0.25, 0.004, 4700),
    (0.012, 0.35, 0.25, 0.004, 4700),
    (0.010, 0.40, 0.25, 0.005, 4680),
    (0.012, 0.40, 0.30, 0.005, 4680),
    (0.008,  0.30, 0.20, 0.003, 4700),
    (0.009,  0.35, 0.25, 0.003, 4720),
    (0.007,  0.28, 0.18, 0.003, 4720),
    (0.007,  0.25, 0.15, 0.003, 4740),
    (0.006,  0.25, 0.15, 0.002, 4740),
]
best = None
for g in grid:
    sc, rmean, failr = eval_policy(g)
    print(f"grid{g}: score={sc:.5f}  R̄={rmean:.3f}  fail~{failr:.3f}")
    if (best is None) or (sc > best[0]):
        best = (sc, g)
print("Best headroom grid:", best)
best_params = best[1]

grid(0.01, 0.3, 0.2, 0.004, 4700): score=0.74366  R̄=0.790  fail~0.000
grid(0.01, 0.35, 0.25, 0.004, 4700): score=0.74356  R̄=0.790  fail~0.000
grid(0.012, 0.35, 0.25, 0.004, 4700): score=0.74283  R̄=0.788  fail~0.000
grid(0.01, 0.4, 0.25, 0.005, 4680): score=0.74329  R̄=0.789  fail~0.000
grid(0.012, 0.4, 0.3, 0.005, 4680): score=0.74256  R̄=0.788  fail~0.000
grid(0.008, 0.3, 0.2, 0.003, 4700): score=0.74409  R̄=0.792  fail~0.000
grid(0.009, 0.35, 0.25, 0.003, 4720): score=0.74360  R̄=0.791  fail~0.000
grid(0.007, 0.28, 0.18, 0.003, 4720): score=0.74456  R̄=0.793  fail~0.000
grid(0.007, 0.25, 0.15, 0.003, 4740): score=0.74463  R̄=0.793  fail~0.000
grid(0.006, 0.25, 0.15, 0.002, 4740): score=0.74469  R̄=0.794  fail~0.000
Best headroom grid: (0.7446869166666668, (0.006, 0.25, 0.15, 0.002, 4740))


In [14]:
# ===================== Cell 7. Fail-Classifier (risk surrogate) — FIXED labels & split =====================
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

# --- 1) Метка риска: (a) доп. раунды   (b) экстремально высокий synErr относительно блока ---
hist_mask = df['_ord_'] < start_ord

nec = df['N_EC_rounds'].fillna(1).astype(int)
syn = df['synErr'].fillna(0).astype(float)

# robust-порог по synErr внутри блока: med + 3*MAD (MAD=median(|x-med|), устойчива к выбросам)
blk_med = syn[hist_mask].groupby(df.loc[hist_mask, 'block_id']).transform('median')
blk_mad = (syn[hist_mask].groupby(df.loc[hist_mask, 'block_id'])
                      .transform(lambda s: (s - s.median()).abs().median()).replace(0, np.nan))
blk_mad = blk_mad.fillna(blk_mad.median())  # на случай констант в блоке

syn_hist = syn.loc[hist_mask].to_numpy()
thr = (blk_med + 3.0*blk_mad).to_numpy()
syn_extreme_hist = (syn_hist > thr).astype(int)

# отправим метку только для истории; на будущем участке она не используется
y_hist = ((nec.loc[hist_mask] >= 2).astype(int).to_numpy() | syn_extreme_hist).astype(int)

# --- 2) Признаки (онлайн-безопасные) ---
cand_base = [
    'E_mu_Z_lag1','E_mu_Z_lag2','E_mu_Z_lag3',
    'E_mu_Z_ema0.33','E_mu_Z_ema0.5','E_mu_Z_std8','E_mu_Z_std16',
    'opticalPower_ema0.33','opticalPower_ema0.5','opticalPower_diff1',
    'maint_prev','synErr_prev','N_EC_prev',
    'frame_sin','frame_cos',
    'E_mu_phys_est_lag1','E_mu_phys_est_lag2','E_mu_phys_est_lag24',
    'polarizerVoltages[2]','polarizerVoltages[3]'
]
clf_base_cols = [c for c in cand_base if c in df.columns]

X_hist = df.loc[hist_mask, clf_base_cols].copy().fillna(0.0)

# Контрольные признаки для маржи Шеннона (как раньше, только по истории)
fec_const = 1.12
def H2(x):
    x = np.clip(x, 1e-12, 1-1e-12)
    return -(x*np.log2(x) + (1-x)*np.log2(1-x))

X_hist['E_proxy']  = df.loc[hist_mask, 'E_mu_Z'].astype(float).values
X_hist['R_hist']   = df.loc[hist_mask, 'R'].astype(float).fillna(0.75).values
X_hist['s_hist']   = df.loc[hist_mask, 's'].astype(float).fillna(2400).values
X_hist['p_hist']   = df.loc[hist_mask, 'p'].astype(float).fillna(2400).values
X_hist['shannon_margin'] = (1.0 - fec_const*H2(X_hist['E_proxy'].values)) - X_hist['R_hist'].values

# --- 3) Стратифицированный train/val сплит по прошлым данным ---
idx_hist = np.where(hist_mask)[0]
# ограничим объём для ускорения, но сохраним баланс
max_train_rows = 120_000
if len(idx_hist) > max_train_rows:
    # возьмём «хвост» истории, который ближе к таргету
    idx_hist = idx_hist[-max_train_rows:]
    X_hist   = X_hist.iloc[-max_train_rows:, :]
    y_hist   = y_hist[-max_train_rows:]

# Проверим, что в выборке есть обе классы; если нет — ослабим правило synErr
if len(np.unique(y_hist)) < 2:
    q90 = np.nanpercentile(syn_hist, 90)
    y_hist = ((nec.loc[hist_mask] >= 2).astype(int).to_numpy() | (syn_hist > q90)).astype(int)

# Теперь сплит
X_fit, X_val, y_fit, y_val = train_test_split(
    X_hist.values, y_hist, test_size=0.15, random_state=2025, stratify=y_hist
)

# Балансировка классов
pos_w = max(1.0, (len(y_fit)-y_fit.sum())/max(1,y_fit.sum()))
w_fit = np.where(y_fit==1, pos_w, 1.0).astype(np.float32)
w_val = np.where(y_val==1, pos_w, 1.0).astype(np.float32)

clf_features = list(X_hist.columns)
train_pool = Pool(X_fit, y_fit, weight=w_fit, feature_names=clf_features)
valid_pool = Pool(X_val, y_val, weight=w_val, feature_names=clf_features)

# --- 4) Обучение классификатора ---
clf = CatBoostClassifier(
    loss_function='Logloss',
    depth=6,
    learning_rate=0.05,
    l2_leaf_reg=6.0,
    random_strength=0.2,
    iterations=1200,
    early_stopping_rounds=200,
    random_seed=2025,
    task_type='CPU',
    verbose=150
)
clf.fit(train_pool, eval_set=valid_pool, use_best_model=True, verbose=150)
print("Fail-classifier trained.")

# Быстрые sanity-метрики
p_val = clf.predict_proba(valid_pool)[:,1]
print("Val mean P(fail):", float(p_val.mean()))
print("Val P(fail) on positives / negatives:",
      float(p_val[y_val==1].mean()) if (y_val==1).any() else np.nan, "/",
      float(p_val[y_val==0].mean()) if (y_val==0).any() else np.nan)

# --- 5) Инференс-хелперы (как раньше) ---
def build_clf_vector(row, E_proxy_val, R, S):
    x = {c: float(row.get(c, 0.0) or 0.0) for c in clf_base_cols}
    x['E_proxy']  = float(E_proxy_val)
    x['R_hist']   = float(R)
    x['s_hist']   = float(S)
    x['p_hist']   = float(4800 - S)
    x['shannon_margin'] = (1.0 - fec_const*H2(x['E_proxy'])) - x['R_hist']
    return np.array([x[c] for c in clf_features], dtype=np.float32).reshape(1, -1)

def fail_prob_predict(E, R, s, row_features):
    vec = build_clf_vector(row_features, E_proxy_val=E, R=R, S=s)
    return float(clf.predict_proba(vec)[0,1])

0:	learn: 0.6420794	test: 0.6427351	best: 0.6427351 (0)	total: 18.2ms	remaining: 21.8s
150:	learn: 0.2165208	test: 0.2206881	best: 0.2206881 (150)	total: 2.71s	remaining: 18.8s
300:	learn: 0.1951628	test: 0.2025633	best: 0.2025633 (300)	total: 5.51s	remaining: 16.4s
450:	learn: 0.1819543	test: 0.1926255	best: 0.1926255 (450)	total: 8.22s	remaining: 13.7s
600:	learn: 0.1720487	test: 0.1854918	best: 0.1854918 (600)	total: 10.9s	remaining: 10.9s
750:	learn: 0.1637275	test: 0.1801858	best: 0.1801858 (750)	total: 13.7s	remaining: 8.18s
900:	learn: 0.1562915	test: 0.1754379	best: 0.1754379 (900)	total: 16.4s	remaining: 5.44s
1050:	learn: 0.1502793	test: 0.1719625	best: 0.1719584 (1049)	total: 19.1s	remaining: 2.71s
1199:	learn: 0.1451740	test: 0.1695967	best: 0.1695967 (1199)	total: 21.8s	remaining: 0us

bestTest = 0.1695966996
bestIteration = 1199

Fail-classifier trained.
Val mean P(fail): 0.7101893536742658
Val P(fail) on positives / negatives: 0.9192122422636733 / 0.19802918790114005


In [15]:
from sklearn.metrics import roc_auc_score, average_precision_score
print("Val AUC:", roc_auc_score(y_val, p_val))
print("Val AP :", average_precision_score(y_val, p_val))

Val AUC: 0.979629177030646
Val AP : 0.9920385684845808


In [16]:
# ===================== Cell 8. f_EC regression (on successful frames) =====================
from catboost import CatBoostRegressor, Pool

# Successful frames: synErr==0 & N_EC_rounds==1
train_mask = df['_ord_'] < start_ord
good_mask  = train_mask & (df['synErr'].fillna(0)==0) & (df['N_EC_rounds'].fillna(1)==1)

def H2(x):
    x = np.clip(x, 1e-12, 1-1e-12)
    return -(x*np.log2(x) + (1-x)*np.log2(1-x))

fec_target = ((1.0 - df.loc[good_mask, 'R'].astype(float)) /
              H2(df.loc[good_mask, 'E_mu_Z'].astype(float)))
fec_target = fec_target.replace([np.inf,-np.inf], np.nan).clip(1.04, 1.25)

fec_feats = [
    c for c in [
        'E_mu_Z_lag1','E_mu_Z_lag2','E_mu_Z_lag3',
        'E_mu_Z_ema0.2','E_mu_Z_ema0.33','E_mu_Z_ema0.5',
        'E_mu_Z_std8','E_mu_Z_std16',
        'opticalPower_ema0.33','opticalPower_ema0.5','opticalPower_diff1',
        'frame_sin','frame_cos','maint_prev','synErr_prev','N_EC_prev',
        'E_mu_phys_est_lag1','E_mu_phys_est_lag2',
        'polarizerVoltages[2]','polarizerVoltages[3]',
        'block_id_str'
    ] if c in df.columns or c=='block_id_str'
]
fec_feats = list(dict.fromkeys(fec_feats))

X_fec = df.loc[good_mask, fec_feats].copy()
X_fec['block_id_str'] = X_fec['block_id_str'].astype('category')
y_fec = fec_target.astype(float).to_numpy()

cat_idx = [X_fec.columns.get_loc('block_id_str')]
fec_model = CatBoostRegressor(
    loss_function='MAE',
    depth=6, iterations=1200, learning_rate=0.05, l2_leaf_reg=3.0,
    random_state=2025, task_type='CPU', verbose=200, early_stopping_rounds=100
)
fec_model.fit(Pool(X_fec, y_fec, cat_features=cat_idx),
              eval_set=Pool(X_fec, y_fec, cat_features=cat_idx),
              use_best_model=True)

# Predict fEC on target
X_fec_t = df.loc[target_idx, fec_feats].copy()
X_fec_t['block_id_str'] = X_fec_t['block_id_str'].astype('category')
fec_hat = fec_model.predict(Pool(X_fec_t, cat_features=[X_fec_t.columns.get_loc('block_id_str')]))
fec_hat = np.clip(fec_hat, 1.05, 1.22)
print("f_EC regression ready. Stats:", float(np.mean(fec_hat)), float(np.min(fec_hat)), float(np.max(fec_hat)))

0:	learn: 0.0397414	test: 0.0397414	best: 0.0397414 (0)	total: 991us	remaining: 1.19s
200:	learn: 0.0003470	test: 0.0007000	best: 0.0007000 (200)	total: 186ms	remaining: 924ms
400:	learn: 0.0000175	test: 0.0004039	best: 0.0004038 (398)	total: 399ms	remaining: 795ms
600:	learn: 0.0000012	test: 0.0003906	best: 0.0003906 (599)	total: 609ms	remaining: 607ms
800:	learn: 0.0000005	test: 0.0003902	best: 0.0003902 (796)	total: 856ms	remaining: 426ms
1000:	learn: 0.0000004	test: 0.0003901	best: 0.0003901 (923)	total: 952ms	remaining: 189ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.0003900910056
bestIteration = 923

Shrink model to first 924 iterations.
f_EC regression ready. Stats: 1.170554499999395 1.0943716332725422 1.22


In [22]:
# ===================== Cell 9. Lambda tuning for risk-aware utility — FAST + PROGRESS =====================
import time
np.seterr(all='ignore')

R_GRID = np.round(np.arange(0.50, 0.901, 0.05), 2)

# --- Параметры быстродействия ---
BT_LEN_LAM     = 2000          # длина хвоста для подбора λ (можешь поднять до 3000/4000)
STEP_LOG       = 300           # как часто печатать прогресс (кадров)
S_FAST_DS      = [0, +120, -120]  # смещения s вокруг s*
MAX_R_SPAN     = 2             # использовать до 2 значений R вокруг базового (вместо 3)

# --- Хвост для λ-подбора ---
bt_idx_lam = bt_idx[-BT_LEN_LAM:] if len(bt_idx) > BT_LEN_LAM else bt_idx
n_lam = len(bt_idx_lam)

# --- Все массивы строго как numpy ---
blocks_b = df.loc[bt_idx_lam, 'block_id'].astype(int).to_numpy()
vol8_b   = df.loc[bt_idx_lam, 'E_mu_Z_std8'].fillna(0.0).to_numpy()
vol16_b  = df.loc[bt_idx_lam, 'E_mu_Z_std16'].fillna(0.0).to_numpy()
trend_b  = (df.loc[bt_idx_lam, 'E_mu_Z_diff1'].fillna(0.0).to_numpy() > 0).astype(float)

maint_b  = (df.loc[bt_idx_lam, 'maintenance_flag'].fillna(0).to_numpy()==1).astype(float)
syn_b    = (df.loc[bt_idx_lam, 'synErr_prev'].fillna(0).to_numpy()>0).astype(float)
nec_b    = (df.loc[bt_idx_lam, 'N_EC_prev'].fillna(0).to_numpy()>1).astype(float)
flags_b  = maint_b + syn_b + nec_b

E_true_b = df.loc[bt_idx_lam, 'E_mu_Z'].astype(float).to_numpy()

# --- fEC на окне подборa λ ---
X_fec_b = df.loc[bt_idx_lam, fec_feats].copy()
X_fec_b['block_id_str'] = X_fec_b['block_id_str'].astype('category')
fec_hat_b = fec_model.predict(Pool(X_fec_b, cat_features=[X_fec_b.columns.get_loc('block_id_str')]))
fec_hat_b = np.clip(fec_hat_b, 1.05, 1.22)

# --- Предвычислим Rt и hr (они не зависят от R/s) ---
def H2(x):
    x = np.clip(x, 1e-12, 1-1e-12)
    return -(x*np.log2(x) + (1-x)*np.log2(1-x))

Rt_b = np.clip(1.0 - fec_hat_b * H2(E_true_b), 0.0, 0.95)
base_head, kv8, kv16, ktrend, s_ceiling = best_params
hr_b  = np.clip(base_head + kv8*vol8_b + kv16*vol16_b + ktrend*(trend_b>0), 0.004, 0.030)
safe_b= np.maximum(0.0, Rt_b - hr_b)

# --- Базовые фичи классификатора для всех кадров (одним куском) ---
base_cols = [c for c in clf_base_cols if c in clf_features]
base_mat  = df.loc[bt_idx_lam, base_cols].fillna(0.0).to_numpy(dtype=float)

# --- Позиции служебных признаков в матрице классификатора ---
pos_map = {name: clf_features.index(name) for name in ['E_proxy','R_hist','s_hist','p_hist','shannon_margin']}
base_pos = np.array([clf_features.index(c) for c in base_cols], dtype=int)
n_feat   = len(clf_features)

fec_const = 1.12

def choose_R_list(j, R_prev):
    # базовый r0 из safe_b[j]
    r_cands = R_GRID[R_GRID <= safe_b[j]]
    r0 = float(r_cands.max()) if len(r_cands) else 0.50

    allow_up2 = (vol8_b[j] < 0.007) and (vol16_b[j] < 0.01) and (trend_b[j] == 0) and (flags_b[j] == 0)
    if R_prev is not None:
        j_prev = int(np.where(R_GRID==np.round(R_prev,2))[0][0])
        j_cur  = int(np.where(R_GRID==np.round(r0,2))[0][0])
        j_cur  = max(min(j_cur, j_prev+(2 if allow_up2 else 1)), j_prev-1)
        r0     = float(R_GRID[j_cur])

    j0   = int(np.where(R_GRID==np.round(r0,2))[0][0])
    j_lo = max(0, j0-1)
    j_hi = min(len(R_GRID)-1, j0+1)
    # Ограничим до MAX_R_SPAN значений (ускорение)
    js = list(range(j_lo, j_hi+1))[:MAX_R_SPAN]
    return [float(R_GRID[k]) for k in js]

def backtest_weights(lmbd):
    lam1, lam2, lam3, lam4 = lmbd
    R_prev = None
    util_sum = r_sum = fail_true_sum = 0.0

    # Предвыделение массивов под кандидатов (максимум 3*|S_FAST_DS| ~ 9)
    maxC = MAX_R_SPAN * len(S_FAST_DS)
    Xcand = np.zeros((maxC, n_feat), dtype=np.float32)

    t0 = time.time()
    for j in range(n_lam):
        # R-кандидаты
        R_list = choose_R_list(j, R_prev)
        # Список кандидатов (R, s, p, Rt)
        C = []
        Rt = Rt_b[j]
        for Rv in R_list:
            s_star = int(np.clip(round(32000*Rv - 27200*Rt), 0, 4800))
            for ds in S_FAST_DS:
                s = int(np.clip(s_star + ds, 0, 4800))
                C.append((Rv, s, 4800 - s, Rt))
        # дедуп
        C = list({(R,s,p):Rt for (R,s,p,Rt) in C}.items())
        C = [(R, s, p, Rt) for (R,s,p), Rt in C]
        Cn = len(C)

        # заполнение батч-матрицы
        Xcand.fill(0.0)
        Xcand[:Cn, base_pos] = base_mat[j][None, :]

        CR  = np.fromiter((c[0] for c in C), dtype=float, count=Cn)
        CS  = np.fromiter((c[1] for c in C), dtype=float, count=Cn)
        CP  = 4800.0 - CS
        CRt = np.fromiter((c[3] for c in C), dtype=float, count=Cn)

        e = float(E_true_b[j])
        sm = (1.0 - fec_const*H2(e)) - CR
        dR = 0.0 if R_prev is None else np.abs(CR - R_prev)

        Xcand[:Cn, pos_map['E_proxy']]         = e
        Xcand[:Cn, pos_map['R_hist']]          = CR
        Xcand[:Cn, pos_map['s_hist']]          = CS
        Xcand[:Cn, pos_map['p_hist']]          = CP
        Xcand[:Cn, pos_map['shannon_margin']]  = sm

        # риск фейла для всех кандидатов сразу
        pf = clf.predict_proba(Xcand[:Cn, :])[:,1]

        # utility
        R_eff  = (32000.0*CR - CS) / 27200.0
        margin = R_eff - CRt
        util   = (CR
                  - lam1*pf
                  - lam2*np.maximum(0.0, margin)
                  - lam3*(CS/4800.0)
                  - lam4*dR)

        k = int(np.argmax(util))
        best_R, best_S = float(CR[k]), int(CS[k])

        # потолок s — понижаем R на ступень, если выгодно
        if best_S >= s_ceiling:
            pos = int(np.where(R_GRID==np.round(best_R,2))[0][0])
            if pos > 0:
                R2 = float(R_GRID[pos-1])
                s2 = int(np.clip(round(32000*R2 - 27200*Rt), 0, 4800))
                if s2 <= (s_ceiling - 60):
                    best_R, best_S = R2, s2

        util_sum     += float(util[k])
        r_sum        += best_R
        fail_true_sum+= float(df.at[bt_idx_lam[j], 'N_EC_rounds'] >= 2)
        R_prev = best_R

        # прогресс
        if (j+1) % STEP_LOG == 0:
            elapsed = time.time() - t0
            print(f"  .. {j+1}/{n_lam} frames for λ={lmbd}, "
                  f"avg util={util_sum/(j+1):.5f}, R̄={r_sum/(j+1):.3f}, "
                  f"fail≈{fail_true_sum/(j+1):.3f}, {elapsed:.1f}s", flush=True)

    return (util_sum/n_lam, r_sum/n_lam, fail_true_sum/n_lam)

# --- сетка λ ---
lambda_grid = [
    (2.0, 1.0, 0.05, 0.02),
    (2.5, 1.2, 0.05, 0.02),
    (3.0, 1.2, 0.06, 0.03),
    (3.5, 1.4, 0.06, 0.03),
    (4.0, 1.6, 0.07, 0.04),
    (4.5, 1.6, 0.07, 0.04),
    (5.0, 1.8, 0.07, 0.05),
]

best_tuple = None
for lambdas in lambda_grid:
    print(f"λ={lambdas} → running backtest on {n_lam} frames ...", flush=True)
    u, rbar, failr = backtest_weights(lambdas)
    print(f"λ={lambdas}: util={u:.5f}  R̄={rbar:.3f}  fail≈{failr:.3f}", flush=True)
    if (best_tuple is None) or (u > best_tuple[0]):
        best_tuple = (u, lambdas)

print("Best λ:", best_tuple[1])
LAM = best_tuple[1]

λ=(2.0, 1.0, 0.05, 0.02) → running backtest on 2000 frames ...
  .. 300/2000 frames for λ=(2.0, 1.0, 0.05, 0.02), avg util=0.07768, R̄=0.771, fail≈0.647, 0.4s
  .. 600/2000 frames for λ=(2.0, 1.0, 0.05, 0.02), avg util=0.06943, R̄=0.766, fail≈0.683, 0.8s
  .. 900/2000 frames for λ=(2.0, 1.0, 0.05, 0.02), avg util=0.04923, R̄=0.768, fail≈0.677, 1.1s
  .. 1200/2000 frames for λ=(2.0, 1.0, 0.05, 0.02), avg util=0.05658, R̄=0.769, fail≈0.682, 1.5s
  .. 1500/2000 frames for λ=(2.0, 1.0, 0.05, 0.02), avg util=0.08445, R̄=0.768, fail≈0.686, 1.9s
  .. 1800/2000 frames for λ=(2.0, 1.0, 0.05, 0.02), avg util=0.07849, R̄=0.770, fail≈0.688, 2.3s
λ=(2.0, 1.0, 0.05, 0.02): util=0.09440  R̄=0.769  fail≈0.683
λ=(2.5, 1.2, 0.05, 0.02) → running backtest on 2000 frames ...
  .. 300/2000 frames for λ=(2.5, 1.2, 0.05, 0.02), avg util=-0.09007, R̄=0.770, fail≈0.647, 0.4s
  .. 600/2000 frames for λ=(2.5, 1.2, 0.05, 0.02), avg util=-0.09933, R̄=0.765, fail≈0.683, 0.8s
  .. 900/2000 frames for λ=(2.5, 1.2, 0.

In [24]:
# ===================== Cell 10. FINAL solver + save submission =====================
R_GRID = np.round(np.arange(0.50, 0.901, 0.05), 2)
base_head, kv8, kv16, ktrend, s_ceiling = best_params

def H2(x):
    x = np.clip(x, 1e-12, 1-1e-12)
    return -(x*np.log2(x) + (1-x)*np.log2(1-x))

def choose_candidates(e, fec, R_prev, v8, v16, trend_up, flags, head_override=None):
    Rt = float(np.clip(1.0 - fec*H2(e), 0.0, 0.95))
    hr = float(np.clip((head_override if head_override is not None
                        else base_head + kv8*v8 + kv16*v16 + ktrend*(trend_up>0)), 0.004, 0.030))
    safe = max(0.0, Rt - hr)

    r_cands = R_GRID[R_GRID <= safe]
    r0 = float(r_cands.max()) if len(r_cands) else 0.50

    allow_up2 = (v8 < 0.007) and (v16 < 0.01) and (trend_up == 0) and (flags == 0)
    if R_prev is not None:
        j_prev = int(np.where(R_GRID==np.round(R_prev,2))[0][0])
        j_cur  = int(np.where(R_GRID==np.round(r0,2))[0][0])
        j_cur  = max(min(j_cur, j_prev+(2 if allow_up2 else 1)), j_prev-1)
        r0     = float(R_GRID[j_cur])

    j0   = int(np.where(R_GRID==np.round(r0,2))[0][0])
    j_lo = max(0, j0-1); j_hi = min(len(R_GRID)-1, j0+1)
    R_list = [float(R_GRID[j]) for j in range(j_lo, j_hi+1)]

    cands = []
    for Rv in R_list:
        s_star = int(np.clip(round(32000*Rv - 27200*Rt), 0, 4800))
        for ds in [0, +60, -60, +120, -120]:
            s = int(np.clip(s_star + ds, 0, 4800))
            p = 4800 - s
            cands.append((Rv, s, p, Rt))
    uniq = {}
    for Rv,s,p,Rt_ in cands:
        uniq[(Rv,s,p)] = Rt_
    return [(R,s,p,Rt_) for (R,s,p),Rt_ in uniq.items()]

def utility_for_candidate(e, fec, R_prev, row_bare, cand, lambdas):
    lam1, lam2, lam3, lam4 = lambdas
    R, s, p, Rt = cand
    R_eff = float((32000*R - s) / 27200.0)
    margin = R_eff - Rt

    pf = fail_prob_predict(E=e, R=R, s=s, row_features=row_bare)
    dR = 0.0 if R_prev is None else abs(R - R_prev)

    util = (R
            - lam1*pf
            - lam2*max(0.0, margin)
            - lam3*(s/4800.0)
            - lam4*dR)
    return util, pf, margin

R_sel, s_sel, p_sel = [], [], []
R_prev = None

for i, idx in enumerate(target_idx):
    row   = df.loc[idx]
    e     = float(E_pred_final[i])
    fec   = float(fec_hat[i])
    v8    = float(row.get('E_mu_Z_std8', 0.0) or 0.0)
    v16   = float(row.get('E_mu_Z_std16', 0.0) or 0.0)
    tr    = float((row.get('E_mu_Z_diff1',0.0) or 0.0) > 0.0)
    flg   = float((row.get('maintenance_flag',0)==1)
                  + (row.get('synErr_prev',0) or 0 > 0)
                  + (row.get('N_EC_prev',0) or 0 > 1))

    cands = choose_candidates(e, fec, R_prev, v8, v16, tr, flg, head_override=None)

    best = None
    for cand in cands:
        u, pf, m = utility_for_candidate(e, fec, R_prev, row, cand, LAM)
        # adaptive pf cap: tighter on trouble frames
        pf_cap = 0.08 + 0.04*np.clip(v8,0,0.03) + 0.02*np.clip(v16,0,0.03) + 0.03*(flg>0)
        if pf <= pf_cap:
            if (best is None) or (u > best[0]):
                best = (u, pf, m, cand)

    # if all over cap, pick overall best
    if best is None:
        for cand in cands:
            u, pf, m = utility_for_candidate(e, fec, R_prev, row, cand, LAM)
            if (best is None) or (u > best[0]):
                best = (u, pf, m, cand)

    R, s, p, _ = best[3]

    # s ceiling safeguard with one-step R downshift if helps
    if s >= s_ceiling:
        pos = int(np.where(R_GRID==np.round(R,2))[0][0])
        if pos>0:
            R2 = float(R_GRID[pos-1])
            Rt = float(np.clip(1.0 - fec*H2(e), 0.0, 0.95))
            s2 = int(np.clip(round(32000*R2 - 27200*Rt), 0, 4800))
            if s2 <= (s_ceiling - 60):
                R, s = R2, s2
                p    = 4800 - s

    R_sel.append(np.round(R,2)); s_sel.append(int(s)); p_sel.append(int(p))
    R_prev = R

# --- Save submission ---
out_path = "/kaggle/working/submission.csv"
sub = pd.DataFrame({
    0: [f"{x:.16f}" for x in E_pred_final],
    1: [str(float(r)) for r in R_sel],
    2: s_sel,
    3: p_sel
})
sub.to_csv(out_path, header=False, index=False)
print("Saved:", out_path)

# --- Final checks ---
loaded = pd.read_csv(out_path, header=None)
assert loaded.shape == (2000,4)
R_GRID = np.round(np.arange(0.50,0.901,0.05),2)
assert np.all(np.isin(np.round(loaded[1].astype(float),2), R_GRID))
assert np.all(loaded[2] + loaded[3] == 4800)
print("OK: submission integrity checks passed")
print(loaded.head(10).to_string(index=False))

Saved: /kaggle/working/submission.csv
OK: submission integrity checks passed
       0    1    2    3
0.025053 0.75 2000 2800
0.021263 0.75 1382 3418
0.020319 0.75 1229 3571
0.019056 0.75 1035 3765
0.020326 0.75 1324 3476
0.026168 0.75 2220 2580
0.026033 0.75 2184 2616
0.025371 0.75 2167 2633
0.028107 0.70 1024 3776
0.022824 0.75 1577 3223
